### Imports

In [1]:
import scipy.io.wavfile as wav
import scipy.signal as signal
from matplotlib import pyplot as plt
from os import listdir
from os.path import isfile, join
import numpy as np

In [2]:
path = "genres_original"
genres = ["blues", "classical", "country", "disco", "hiphop", "jazz", "metal", "pop", "reggae", "rock"]
save_path = "genres_images"
max_per_genre = 30

for genre in genres:
    files = [f for f in listdir(path + "/" + genre) if isfile(join(path + "/" + genre, f))]
    n=0
    for file in files:
        sample_rate, samples = wav.read(path + "/" + genre + "/" + file)
        f, t, Zxx = signal.stft(samples.T[0], fs=sample_rate)
        plt.pcolormesh(t, f, np.log(np.abs(Zxx)), cmap='viridis')
        plt.savefig(save_path + "/" + genre + "/" + file[:-4] + '.png', bbox_inches='tight')
        n+=1
        if(n>=30):
            break
    
    


KeyboardInterrupt



Error in callback <function flush_figures at 0x000002140046BEC0> (for post_execute), with arguments args (),kwargs {}:



KeyboardInterrupt

